# import

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import string
import re
from sklearn.utils import resample
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import layers
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.layers import Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/twitter-2016train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/twitter-2016devtest-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df_train = df_train.drop(labels=["id"],axis=1)
df_test = df_test.drop(labels=["id"],axis=1)
df_train.head()

,sentiment,tweets
0,negative,dear @Microsoft the newOoffice for Mac is grea...
1,negative,@Microsoft how about you make a system that do...
2,negative,I may be ignorant on this issue but... should ...
3,negative,"Thanks to @microsoft, I just may be switching ..."
4,neutral,If I make a game as a #windows10 Universal App...


In [ ]:
def label_encoder(x):
    if x == 'positive': return 2
    elif x == 'negative': return 0
    else: return 1
    
newdf = pd.concat([df_train,df_test]).reset_index(drop=True)
newdf.head()



,sentiment,tweets
0,negative,dear @Microsoft the newOoffice for Mac is grea...
1,negative,@Microsoft how about you make a system that do...
2,negative,I may be ignorant on this issue but... should ...
3,negative,"Thanks to @microsoft, I just may be switching ..."
4,neutral,If I make a game as a #windows10 Universal App...


# Preprocessing

# StopWord Remove, Punctuation Remove, Digram TFIDF

In [ ]:
def remove_punctuation(x):
    return x.translate(str.maketrans('', '', string.punctuation))

X = newdf.tweets.apply(lambda x: remove_punctuation(x))
X.head()

0    dear Microsoft the newOoffice for Mac is great...
1    Microsoft how about you make a system that doe...
2    I may be ignorant on this issue but should we ...
3    Thanks to microsoft I just may be switching ov...
4    If I make a game as a windows10 Universal App ...
Name: tweets, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(stop_words = {'english'},ngram_range=(1,3))
X = vectorizer.fit_transform(X)
Y = newdf.sentiment.apply(lambda x: label_encoder(x))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, shuffle = True)

In [ ]:
LR_model = LogisticRegression(max_iter=6000).fit(X_train,Y_train)
SVM_model = SVC(kernel='rbf').fit(X_train, Y_train)
NB_model = ComplementNB().fit(X_train,Y_train)

Result

In [ ]:
print("LR Score  \n", classification_report(Y_test, LR_model.predict(X_test)))


LR Score  
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       242
           1       0.51      0.27      0.35       557
           2       0.57      0.94      0.71       775

    accuracy                           0.56      1574
   macro avg       0.36      0.40      0.35      1574
weighted avg       0.46      0.56      0.47      1574



In [ ]:
print("SVC Score  \n", classification_report(Y_test, SVM_model.predict(X_test)))

SVC Score  
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       242
           1       0.59      0.10      0.16       557
           2       0.52      0.99      0.68       775

    accuracy                           0.52      1574
   macro avg       0.37      0.36      0.28      1574
weighted avg       0.46      0.52      0.39      1574



In [ ]:
print("NB Score  \n", classification_report(Y_test, NB_model.predict(X_test)))

NB Score  
               precision    recall  f1-score   support

           0       0.46      0.14      0.21       242
           1       0.55      0.28      0.37       557
           2       0.58      0.91      0.71       775

    accuracy                           0.57      1574
   macro avg       0.53      0.44      0.43      1574
weighted avg       0.55      0.57      0.51      1574



# Preporocessing-2

# Removes punctuation, stopwords and Dataset resampled for balancing.

In [ ]:
cdf = newdf.copy().reset_index(drop=True)
cdf.head()

,sentiment,tweets
0,negative,dear @Microsoft the newOoffice for Mac is grea...
1,negative,@Microsoft how about you make a system that do...
2,negative,I may be ignorant on this issue but... should ...
3,negative,"Thanks to @microsoft, I just may be switching ..."
4,neutral,If I make a game as a #windows10 Universal App...


In [ ]:
def clean_text(x):
    x = re.sub(r'https?://\S+', '', x) # Remove link
    x = re.sub(r'#\w+', '', x) # Remove hashtag
    x = re.sub(r'@\w+', '', x) # Remove mentions
    x = re.sub(r'\n',' ',x) # Remove line breaks
    x = re.sub('\s+', ' ', x).strip() # Remove leading, trailing, and extra spaces
    x = re.sub('\.','',x) # Remove extra dots
    for p in string.punctuation:
        x = re.sub('\{}'.format(p),'',x)
    return x.lower()

In [ ]:
cdf["final"] = cdf['tweets'].apply(lambda x: clean_text(x))

In [ ]:
cdf.head()

,sentiment,tweets,final
0,negative,dear @Microsoft the newOoffice for Mac is grea...,dear the newooffice for mac is great and all b...
1,negative,@Microsoft how about you make a system that do...,how about you make a system that doesnt eat my...
2,negative,I may be ignorant on this issue but... should ...,i may be ignorant on this issue but should we ...
3,negative,"Thanks to @microsoft, I just may be switching ...",thanks to i just may be switching over to
4,neutral,If I make a game as a #windows10 Universal App...,if i make a game as a universal app will owner...


In [ ]:
cdf['sentiment'] = cdf['sentiment'].apply(lambda x: label_encoder(x))
cdf['sentiment'].value_counts()

2    4011
1    2682
0    1175
Name: sentiment, dtype: int64

In [ ]:
cdf_positive = cdf[(cdf['sentiment']==2)]
cdf_neutral = cdf[(cdf['sentiment']==1)]
cdf_negative = cdf[(cdf['sentiment']==0)]
cdf_neutral_upsampled = resample(cdf_neutral, replace = True, n_samples = 4011, random_state = 1)
cdf_negative_upsampled = resample(cdf_negative, replace = True, n_samples = 4011, random_state = 1)

balancedDataset = pd.concat([cdf_positive,cdf_neutral_upsampled,cdf_negative_upsampled]).reset_index(drop=True)
balancedDataset['sentiment'].value_counts()

2    4011
1    4011
0    4011
Name: sentiment, dtype: int64

In [ ]:

vectorizer = TfidfVectorizer(stop_words = {'english'},ngram_range=(1,2))
X = vectorizer.fit_transform(balancedDataset['final'])
Y = balancedDataset.sentiment
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, shuffle = True)

In [ ]:
LR_model = LogisticRegression(max_iter=13000).fit(X_train,Y_train)
SVM_model = SVC(kernel='rbf', random_state = 1).fit(X_train, Y_train)
NB_model= ComplementNB().fit(X_train,Y_train)

In [ ]:
print("LR Score  \n", classification_report(Y_test, LR_model.predict(X_test)))


LR Score  
               precision    recall  f1-score   support

           0       0.89      0.93      0.91       792
           1       0.78      0.76      0.77       822
           2       0.76      0.75      0.75       793

    accuracy                           0.81      2407
   macro avg       0.81      0.81      0.81      2407
weighted avg       0.81      0.81      0.81      2407



In [ ]:
print("SVC Score  \n", classification_report(Y_test, SVM_model.predict(X_test)))

SVC Score  
               precision    recall  f1-score   support

           0       0.99      0.93      0.96       792
           1       0.90      0.75      0.82       822
           2       0.75      0.92      0.83       793

    accuracy                           0.87      2407
   macro avg       0.88      0.87      0.87      2407
weighted avg       0.88      0.87      0.87      2407



In [ ]:
print("NB Score  \n", classification_report(Y_test, NB_model.predict(X_test)))

NB Score  
               precision    recall  f1-score   support

           0       0.79      0.96      0.87       792
           1       0.83      0.74      0.78       822
           2       0.80      0.72      0.76       793

    accuracy                           0.81      2407
   macro avg       0.81      0.81      0.80      2407
weighted avg       0.81      0.81      0.80      2407



# Final Neural Network Model

In [ ]:
balancedDataset['final'].apply(lambda x:len(str(x).split())).max()

1056

In [ ]:
Y = pd.get_dummies(balancedDataset['sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (12033, 3)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(balancedDataset['final'], Y, train_size=.8, random_state=42)

In [ ]:
maxlen = 1100

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(balancedDataset['final'])

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
def create_embedding_matrix(filepath, word_idx, embedding_dim):
    vocab_size = len(word_idx) + 1  
    emb = np.zeros((vocab_size, embedding_dim))

    with open(filepath, 'r', encoding="utf8") as f:
        for i in f:
            x = i.split(' ')
            w, v = x[0], x[1:]
            if w in word_idx:
                emb[word_idx[w]] = np.asarray(v)[:embedding_dim]

    return emb

In [ ]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('/content/drive/MyDrive/Colab Notebooks/dataset/glove.42B.300d.txt', tokenizer.word_index, embedding_dim)

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                            weights=[embedding_matrix], 
                            input_length=maxlen, 
                            trainable=False))
model.add(Bidirectional(LSTM(300,activation='tanh', dropout=0.1, recurrent_dropout=0)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1100, 300)         4052400   
                                                                 
 bidirectional (Bidirectiona  (None, 600)              1442400   
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 600)               0         
                                                                 
 dense (Dense)               (None, 200)               120200    
                                                                 
 dense_1 (Dense)             (None, 3)                 603       
                                                                 
Total params: 5,615,603
Trainable params: 1,563,203
Non-trainable params: 4,052,400
______________________________________

In [ ]:
epochs = 20
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/20
136/136 [==============================] - 45s 271ms/step - loss: 0.9278 - accuracy: 0.5406 - val_loss: 0.9046 - val_accuracy: 0.5379
Epoch 2/20
136/136 [==============================] - 37s 275ms/step - loss: 0.7979 - accuracy: 0.6341 - val_loss: 0.8760 - val_accuracy: 0.5670
Epoch 3/20
136/136 [==============================] - 38s 279ms/step - loss: 0.6779 - accuracy: 0.7001 - val_loss: 0.7610 - val_accuracy: 0.6739
Epoch 4/20
136/136 [==============================] - 38s 283ms/step - loss: 0.5455 - accuracy: 0.7707 - val_loss: 0.6250 - val_accuracy: 0.7466
Epoch 5/20
136/136 [==============================] - 39s 287ms/step - loss: 0.4119 - accuracy: 0.8296 - val_loss: 0.5599 - val_accuracy: 0.7747
Epoch 6/20
136/136 [==============================] - 40s 292ms/step - loss: 0.2969 - accuracy: 0.8776 - val_loss: 0.5541 - val_accuracy: 0.7985
Epoch 7/20
136/136 [==============================] - 40s 294ms/step - loss: 0.2148 - accuracy: 0.9165 - val_loss: 0.6513 - val_ac

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

76/76 [==============================] - 5s 60ms/step - loss: 0.6533 - accuracy: 0.8330
Test set
  Loss: 0.653
  Accuracy: 0.833
